# Berlin Drugstore

### Loading initial dataset from CSV file to pandas. Cell is hidden because it contains my credentials for Watson studio:

In [1]:
# The code was removed by Watson Studio for sharing.

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376
1,1,Mitte,Moabit,69425,22.0,52.530102,13.342542
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260
4,4,Mitte,Wedding,76363,22.0,52.550123,13.341970


### Installing and importing Geocoder package

In [2]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 526.61 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 802.41 kB/s


### Getting Berlin coordinates:

In [3]:
address = 'Berlin, Germany'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Berlin are 52.5170365, 13.3888599.


### Installing and importing Folium. We need it for map generation:

In [4]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   3.17 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  16.35 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  28.86 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  10.63 MB/s


In [6]:
# create map of Berlin using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['Borough'], df_data_1['Neigborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin

### Another hidden cell. Contains my Foursquare credentials:-(

In [7]:
# The code was removed by Watson Studio for sharing.

### Defining function that will get data from Foursquare:

In [8]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [11]:
import requests
berlin_venues = getNearbyVenues(names=df_data_1['Neigborhood'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )

Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weißensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Spandau
Haselhorst
Siemensstadt
Staaken
Gatow
Kladow
Hakenfelde
Falkenhagener Feld
Wilhelmstadt
Steglitz
Lichterfelde
Lankwitz
Zehlendorf
Dahlem
Nikolassee
Wannsee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt
Alt-Treptow
Plänterwald
Baumschulenweg
Johannisthal
Niederschöneweide
Altglienicke
Adlershof
Bohnsdorf
Oberschöneweide
Köpenick
Friedrichshagen
Rahnsdorf
Grünau
Müggelheim
Schmöckwitz
Marzahn
Biesdorf
Kaulsdorf
Mahlsdorf
Hellersdorf
Friedrichsfelde
Karlshorst
Lichtenberg
Falkenberg
Malchow
Wartenberg
Neu-Hohenschönhausen
Alt-Hohenschönhausen
Fennpfuhl
Rummelsburg
Reinickendorf
Tegel
Kon

In [12]:
print(berlin_venues.shape)
berlin_venues.head()

(1436, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Mitte,52.51769,13.402376,Lustgarten,52.518469,13.399454,Garden
1,Mitte,52.51769,13.402376,Radisson Blu,52.519623,13.402488,Hotel
2,Mitte,52.51769,13.402376,DDR Museum,52.519404,13.402239,History Museum
3,Mitte,52.51769,13.402376,Atrium Lobby Lounge & Bar,52.519597,13.402774,Hotel Bar
4,Mitte,52.51769,13.402376,Designpanoptikum - surreales Museum für indust...,52.516941,13.406072,Museum


In [13]:
berlin_grouped_df = berlin_venues.groupby('Neighborhood').count()
berlin_grouped_df.head()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adlershof,7,7,7,7,7,7
Alt-Hohenschönhausen,7,7,7,7,7,7
Alt-Treptow,31,31,31,31,31,31
Altglienicke,1,1,1,1,1,1
Baumschulenweg,5,5,5,5,5,5


In [14]:
print('There are {} uniques categories.'.format(len(berlin_venues['Venue Category'].unique())))

There are 228 uniques categories.


In [15]:
# one hot encoding
berlin_onehot = pd.get_dummies(berlin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
berlin_onehot['Neighborhood'] = berlin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [berlin_onehot.columns[-1]] + list(berlin_onehot.columns[:-1])
berlin_onehot = berlin_onehot[fixed_columns]
berlin_onehot.head()

,Zoo Exhibit,ATM,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Waterfront,Windmill,Wine Bar,Wine Shop,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
berlin_onehot.shape

(1436, 228)

In [17]:
berlin_grouped = berlin_onehot.groupby('Neighborhood').mean().reset_index()
berlin_grouped

,Neighborhood,Zoo Exhibit,ATM,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Water Park,Waterfront,Windmill,Wine Bar,Wine Shop,Yoga Studio
0,Adlershof,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,Alt-Hohenschönhausen,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.142857,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
2,Alt-Treptow,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.032258,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,Altglienicke,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
4,Baumschulenweg,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
5,Biesdorf,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.083333,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
6,Blankenburg,0.000000,0.200000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
7,Blankenfelde,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
8,Bohnsdorf,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
9,Borsigwalde,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000


### Defining function that will return most common venues in Heighborhood:

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Searching for 20 most common venues in each Neigdborhood: 

In [19]:
import numpy as np
num_top_venues = 20

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = berlin_grouped['Neighborhood']

for ind in np.arange(berlin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(berlin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,Adlershof,Steakhouse,Italian Restaurant,Tram Station,Trattoria/Osteria,Home Service,Greek Restaurant,Insurance Office,Food & Drink Shop,Food Court,...,Flower Shop,Fish Market,Fish & Chips Shop,Fountain,French Restaurant,Fast Food Restaurant,Farm,Fondue Restaurant,Ethiopian Restaurant,Exhibit
1,Alt-Hohenschönhausen,Indian Restaurant,Post Office,Discount Store,Supermarket,Asian Restaurant,Drugstore,Coffee Shop,Yoga Studio,Fondue Restaurant,...,Fish Market,Flower Shop,Food Court,Food & Drink Shop,Farm,Fountain,French Restaurant,Fast Food Restaurant,Event Space,Falafel Restaurant
2,Alt-Treptow,Bus Stop,Platform,Bakery,Café,Pier,Seafood Restaurant,Deli / Bodega,Snack Place,Mexican Restaurant,...,Sandwich Place,Fast Food Restaurant,Garden Center,Nightclub,Light Rail Station,Tapas Restaurant,Big Box Store,Rental Car Location,Beer Garden,Bank
3,Altglienicke,Electronics Store,Yoga Studio,Farm,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Court,Food & Drink Shop,...,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Falafel Restaurant,Deli / Bodega,Exhibit,Event Space,Ethiopian Restaurant,Eastern European Restaurant
4,Baumschulenweg,Supermarket,Ice Cream Shop,Drugstore,Shipping Store,Falafel Restaurant,French Restaurant,Fountain,Food Court,Food & Drink Shop,...,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farm,Exhibit,Frozen Yogurt Shop,Event Space,Ethiopian Restaurant,Electronics Store
5,Biesdorf,Supermarket,Light Rail Station,Palace,Outdoor Event Space,Plaza,Pharmacy,Pet Store,Shipping Store,Athletics & Sports,...,Park,Farm,Fast Food Restaurant,Yoga Studio,Fish & Chips Shop,Falafel Restaurant,Flower Shop,Fondue Restaurant,Food & Drink Shop,Fish Market
6,Blankenburg,ATM,Bus Stop,Café,Flower Shop,Greek Restaurant,Yoga Studio,Fast Food Restaurant,Fried Chicken Joint,French Restaurant,...,Food Court,Food & Drink Shop,Fondue Restaurant,Fish Market,Fish & Chips Shop,Farm,Furniture / Home Store,Falafel Restaurant,Exhibit,Event Space
7,Blankenfelde,Café,Miscellaneous Shop,Yoga Studio,Fast Food Restaurant,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Food Court,...,Fondue Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Farm,Gaming Cafe,Falafel Restaurant,Exhibit,Event Space,Ethiopian Restaurant
8,Bohnsdorf,Insurance Office,Italian Restaurant,Flower Shop,Park,Yoga Studio,Falafel Restaurant,Fried Chicken Joint,French Restaurant,Fountain,...,Food & Drink Shop,Fondue Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farm,Exhibit,Furniture / Home Store,Event Space,Ethiopian Restaurant
9,Borsigwalde,Bakery,Italian Restaurant,Mexican Restaurant,Go Kart Track,Farm,Fried Chicken Joint,French Restaurant,Fountain,Food Court,...,Fondue Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Yoga Studio,Furniture / Home Store,Falafel Restaurant,Exhibit,Event Space


### When venues were requested from Foursquare one Heighborhood is lost, eg. no data were fetched for is so it will ne removed from furhter analysis:

In [20]:
lost = []
for e in df_data_1['Neigborhood'].values:
    if e not in neighborhoods_venues_sorted['Neighborhood'].values:
        lost.append(e)
lost


['Rahnsdorf']

In [21]:
df = df_data_1[df_data_1["Neigborhood"] != 'Rahnsdorf']

In [22]:
df.shape

(95, 7)

In [23]:
df.head()

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376
1,1,Mitte,Moabit,69425,22.0,52.530102,13.342542
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260
4,4,Mitte,Wedding,76363,22.0,52.550123,13.341970


In [28]:
#berlin_merged =df.reset_index(drop=True)


berlin_merged = df.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neigborhood')

berlin_merged.head() # check the last columns!

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376,German Restaurant,Museum,Hotel,...,Concert Hall,Steakhouse,Board Shop,Breakfast Spot,Bookstore,Nightclub,Brewery,Event Space,River,Scenic Lookout
1,1,Mitte,Moabit,69425,22.0,52.530102,13.342542,Café,Breakfast Spot,Supermarket,...,Vegetarian / Vegan Restaurant,Cocktail Bar,Italian Restaurant,Doner Restaurant,Seafood Restaurant,French Restaurant,Food Court,Peruvian Restaurant,Fish Market,Restaurant
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873,Pedestrian Plaza,Art Museum,Café,...,Rental Car Location,Theater,Convenience Store,Plaza,Playground,Pier,Currywurst Joint,Park,Light Rail Station,Pastry Shop
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260,Hotel Bar,Scandinavian Restaurant,Park,...,Dive Bar,Fountain,Food Court,Food & Drink Shop,Fondue Restaurant,Diner,Flower Shop,Discount Store,Fish Market,Fish & Chips Shop
4,4,Mitte,Wedding,76363,22.0,52.550123,13.341970,Ice Cream Shop,Supermarket,Bar,...,Pharmacy,Food & Drink Shop,Fast Food Restaurant,Fondue Restaurant,Flower Shop,Food Court,Fish Market,Fountain,French Restaurant,Fish & Chips Shop


### Finding Neigborhoods with Drugstores in top 20 venues:

In [30]:
ind=[]

for row in neighborhoods_venues_sorted.iterrows():
    index, data = row
    if 'Drugstore' in data.tolist():
        ind.append(index)
print(len(ind))
ind

28


[1,
 4,
 5,
 11,
 13,
 17,
 18,
 19,
 21,
 22,
 26,
 31,
 36,
 37,
 39,
 45,
 46,
 51,
 58,
 61,
 66,
 71,
 75,
 77,
 82,
 84,
 89,
 94]

In [31]:
neighborhoods_venues_sorted.shape

(95, 21)

### And droping those Neighororhoods in order to get only ones with no Drugstores within top 20 venues

In [32]:
no_drugstores = berlin_merged.drop(berlin_merged.index[ind])
no_drugstores.shape

(67, 27)

In [35]:
no_drugstores.head()

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,...,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue,16th Most Common Venue,17th Most Common Venue,18th Most Common Venue,19th Most Common Venue,20th Most Common Venue
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376,German Restaurant,Museum,Hotel,...,Concert Hall,Steakhouse,Board Shop,Breakfast Spot,Bookstore,Nightclub,Brewery,Event Space,River,Scenic Lookout
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873,Pedestrian Plaza,Art Museum,Café,...,Rental Car Location,Theater,Convenience Store,Plaza,Playground,Pier,Currywurst Joint,Park,Light Rail Station,Pastry Shop
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260,Hotel Bar,Scandinavian Restaurant,Park,...,Dive Bar,Fountain,Food Court,Food & Drink Shop,Fondue Restaurant,Diner,Flower Shop,Discount Store,Fish Market,Fish & Chips Shop
6,6,Friedrichshain-Kreuzberg,Friedrichshain,114050,16.0,52.512215,13.450290,Café,Pub,Coffee Shop,...,Plaza,Jewelry Store,Skate Park,Burrito Place,Sauna / Steam Room,Kebab Restaurant,Korean Restaurant,Boarding House,Yoga Studio,Wine Shop
7,7,Friedrichshain-Kreuzberg,Kreuzberg,147227,16.0,52.497644,13.411914,Café,Bar,Turkish Restaurant,...,Breakfast Spot,Music Store,Bistro,Kebab Restaurant,Spanish Restaurant,Boat or Ferry,Playground,Burger Joint,Supermarket,Seafood Restaurant


In [36]:
df_data_1.head()

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376
1,1,Mitte,Moabit,69425,22.0,52.530102,13.342542
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260
4,4,Mitte,Wedding,76363,22.0,52.550123,13.341970


In [37]:
df1 = df_data_1[df_data_1["Neigborhood"].isin(no_drugstores['Neigborhood'].values)]

In [38]:
df1.head()

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
0,0,Mitte,Mitte,79582,22.0,52.517690,13.402376
2,2,Mitte,Hansaviertel,5889,22.0,52.519123,13.341873
3,3,Mitte,Tiergarten,12486,22.0,52.509778,13.357260
6,6,Friedrichshain-Kreuzberg,Friedrichshain,114050,16.0,52.512215,13.450290
7,7,Friedrichshain-Kreuzberg,Kreuzberg,147227,16.0,52.497644,13.411914


In [44]:
max_price = np.median(df_data_1["Price(eur/m2)"].unique())

### To find best Neighborhoods we will filter this dataframe by population(only Neighborhoods with population over 50000 will be considered) and by price of office space per square meter(only Neighborhoods with price liwer than median in Berlin).

In [45]:
matched_df = df1[df1['Population']>50000]
matched_df = matched_df[matched_df['Price(eur/m2)']<max_price]
matched_df

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
44,44,Tempelhof-Schöneberg,Schöneberg,116743,10.0,52.482157,13.355190
50,50,Neukölln,Neukölln,154127,11.0,52.481150,13.435350
64,64,Treptow-Köpenick,Köpenick,59201,11.0,52.453910,13.576413
70,70,Marzahn-Hellersdorf,Marzahn,102398,10.0,52.542948,13.563142
74,74,Marzahn-Hellersdorf,Hellersdorf,72602,10.0,52.536854,13.604774
75,75,Lichtenberg,Friedrichsfelde,50010,8.5,52.502936,13.520546
81,81,Lichtenberg,Neu-Hohenschönhausen,53698,8.5,52.566331,13.514065


### Matched Neighborhoods sorted by Price:

In [51]:
matched_df.sort_values(by = ['Price(eur/m2)', 'Population'])

,Unnamed: 0,Borough,Neigborhood,Population,Price(eur/m2),Latitude,Longitude
75,75,Lichtenberg,Friedrichsfelde,50010,8.5,52.502936,13.520546
81,81,Lichtenberg,Neu-Hohenschönhausen,53698,8.5,52.566331,13.514065
74,74,Marzahn-Hellersdorf,Hellersdorf,72602,10.0,52.536854,13.604774
70,70,Marzahn-Hellersdorf,Marzahn,102398,10.0,52.542948,13.563142
44,44,Tempelhof-Schöneberg,Schöneberg,116743,10.0,52.482157,13.355190
64,64,Treptow-Köpenick,Köpenick,59201,11.0,52.453910,13.576413
50,50,Neukölln,Neukölln,154127,11.0,52.481150,13.435350


### And here is ta map of matched Neigdborhoods:

In [47]:
# create map of New York using latitude and longitude values
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(matched_df['Latitude'], matched_df['Longitude'], matched_df['Borough'], matched_df['Neigborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin)  
    
map_berlin